# Order Nakeds

In [ ]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = "NSE"

# Set the root
from from_root import from_root
ROOT = from_root()

import pandas as pd
from loguru import logger

pd.options.display.max_columns = None
pd.set_option('display.precision', 2)

from pathlib import Path
import sys

# Add `src` and ROOT to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path

if "src" not in Path.cwd().parts:
    src_path = str(Path(get_path("purelib")) / "_src.pth")
    with open(src_path, "w") as f:
        f.write(str(ROOT / "src\n"))
        f.write(str(ROOT))
        if str(ROOT) not in sys.path:
            sys.path.insert(1, str(ROOT))

# Start the Jupyter loop
from ib_async import util

util.startLoop()

logger.add(sink=ROOT / "log" / "ztest.log", mode="w")

## Imports

In [ ]:
from utils import handle_raws, get_pickle, load_config, arrange_orders, pickle_me, get_file_age, yes_or_no
from datetime import datetime
from ib_async import IB
from ibfuncs import get_open_orders, quick_pf, place_orders, make_ib_orders

## Set constants

In [ ]:
config = load_config()
port = config.get('PORT')
MARGINPERORDER = config.get('MARGINPERORDER')

## Handle Raws

In [ ]:
# Consolidate raw files to df_nakeds.pkl
handle_raws()

In [ ]:
file_path = ROOT / 'data' / 'df_nakeds.pkl'

def how_many_days_old(file_path) -> float:
    """Gets the file's age in days"""
    file_age = get_file_age(file_path=file_path)
    
    seconds_in_a_day = 86400
    file_age_in_days = file_age.td.total_seconds() / seconds_in_a_day if file_age else 0
    
    return file_age_in_days

In [ ]:
## Check the age of df_pickles, before ordering

In [ ]:
txt = f"df_nakeds.pkl is {how_many_days_old(file_path): 0.2f}. Want to load?"
ans = yes_or_no(txt)

if ans:
    df_opts = get_pickle(file_path)
    print('\n\n')
    print(df_opts.drop(columns=['nse_symbol', 'instrument', 'contract', 'expiry']).head())
else:
    print('Bye!!!')

## check open orders

## Arrange and make orders

## PLACE THE ORDER

# Archive the orders into `xn_history`

In [ ]:
from ibfuncs import order_nakeds
order_nakeds(df_opts=df_opts, port=port)